# Decision Trees

- Decision trees are non-parametric models
- It uses recursive binary splitting to grow a regression/classification tree
- Decision trees are greedy which only find the best split on the corresponding region at current timestep
- In regression trees, RSS is used to find the best split
- In classification trees, impurity is used to find the best split
- Best node is the feature+split pair with least RSS or impurity at current region

## Regression

Regression trees use RSS (Residual Sums of Squares) to split the current data space.

$RSS = \sum_i(y_i-y_{r1})^2 + \sum_i(y_i-y_{r2})^2$ <br><br>
where $y_{r1}$=average value of region 1, and $y_{r2}$=average value of region 2


## Classification
Classification trees use impurity to split the current data space. Common impurity measures are gini index, entropy and information gain.

$GI = 1 - \sum_j(p_j)^2$ <br>
$entropy = 1 - \sum_j(p_j*log(p_j))$ <br><br>
where j=target class <br><br>

impurity of current node = weight_1 * impurity_of_region_1 + weight_2 * impurity_of_region_2<br>
weight = num_of_sample_in_the_region/total_num_of_sample_in_parent